In [ ]:
import pandas as pd

: 

In [ ]:
# Define file pathname
base = "/Users/ziadharmanani/Documents/GitHub/INF1340-Group7/INF1340-Data/Bike share ridership "
extension = [
    "2023-01", "2023-02", "2023-03", "2023-04", "2023-05", "2023-06",
    "2023-07", "2023-08", "2023-09", "2023-10", "2023-11", "2023-12",
    ]

# Define columns order in a list
columns = [
    "Trip Id", "Trip  Duration", "Start Station Id", "Start Time",
    "Start Station Name", "End Station Id", "End Time",
    "End Station Name", "Bike Id", "User Type"
    ]

# Used to store file paths
paths = []

# Loop through file extensions and combine them to locate them individually
for month in extension:
    path = base + month + ".csv"
    paths.append(path)

# Used to store the individual data frames
all_dfs = []

# Clean columns (Inconsistencies between 2024 and other years)
# Mismatch beteen the years issued a lot of NaN values. Trip Id was duplicated (2024 had its own).
for file in paths:
    df_temp = pd.read_csv(file, encoding = "latin1")
    df_temp.columns = [col.replace("ï»¿", "").strip() for col in df_temp.columns]

    # Used to clean BOM character for "Trip Id" (2024)
    if "ï»¿Trip Id" in df_temp.columns:
        df_temp["Trip Id"] = df_temp["ï»¿Trip Id"]
        df_temp = df_temp.drop(columns = ["ï»¿Trip Id"])
    
    # Reindexes the data frame according to columns list.
    df_temp = df_temp.reindex(columns = columns)
    all_dfs.append(df_temp)

# Concatinate dataframe
df = pd.concat(all_dfs, ignore_index = True)

# Drop "Model" to stay consistent with previous years (Only present in 2024)
df = df.drop(columns = ["Model", "Bike Id"], errors = "ignore")

#remove extra space in Trip Duration column
#df.columns = df.columns.str.replace("Trip  Duration", "Trip Duration")

# NEXT STEPS: Look into Start Station Name and End Station Name.


In [ ]:
df.sample(10)

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,User Type
276932,20465705,1014,7600,02/15/2023 11:10,Russell St / Huron St - SMART,7524.0,02/15/2023 11:27,Lisgar Park,Casual Member
5608545,26566226,539,7475,12/15/2023 16:04,Jarvis St / Richmond St E,7299.0,12/15/2023 16:13,Mill St / Parliament St,Casual Member
1373255,21733201,619,7031,05/24/2023 15:40,Jarvis St / Isabella St,7531.0,05/24/2023 15:50,541 Huron St - SMART,Casual Member
1589122,21981049,1059,7028,06/02/2023 20:09,Gould St / Mutual St,7100.0,06/02/2023 20:27,Dundas St E / Regent Park Blvd,Annual Member
215769,20396091,414,7006,02/07/2023 17:46,Bay St / College St (East Side),7281.0,02/07/2023 17:53,Charles St W / Balmuto St - SMART,Casual Member
353604,20555377,499,7054,03/01/2023 08:15,Navy Wharf Ct. / Bremner Blvd.,7076.0,03/01/2023 08:23,York St / Queens Quay W,Casual Member
5471343,26412994,146,7039,12/02/2023 21:16,Simcoe St / Michael Sweet Ave,7542.0,12/02/2023 21:19,Queen St W / John St,Casual Member
5008351,25889235,904,7601,10/28/2023 14:49,Brick Works,7452.0,10/28/2023 15:04,Bleecker St / St James Ave,Casual Member
4204614,24973630,1216,7313,09/20/2023 19:34,Coxwell Ave / Lake Shore Blvd E,7346.0,09/20/2023 19:54,Logan Av / Gerrard St E,Casual Member
711974,20964278,368,7170,04/12/2023 20:16,Spadina Ave / Willcocks St,7235.0,04/12/2023 20:22,Bay St / College St (West Side) - SMART,Casual Member


In [ ]:
print(f"Start Station Name: {df['Start Station Name'].nunique()}")
print(f"End Station Name: {df['End Station Name'].nunique()}")

Start Station Name: 593
End Station Name: 592


In [ ]:
df.isna().sum()

Trip Id                    0
Trip  Duration             0
Start Station Id           0
Start Time                 0
Start Station Name    595075
End Station Id          2944
End Time                   0
End Station Name      598563
User Type                  0
dtype: int64

In [ ]:
#split start time into 2 columns
#df[['Start Date', 'Start Time']] = df['Start Time'].str.split(' ', expand=True)

#move start date to the 3rd index
col = df.pop('Start Date')
df.insert(3, col.name, col)

#convert date to datetime format
df['Start Date'] = pd.to_datetime(df['Start Date'], format = '%m/%d/%Y')

In [ ]:
#split date into day/month but keep og column
df['Start Day'] = df['Start Date'].dt.day

df['Start Month'] = df['Start Date'].dt.month

#move start day to the 4th index
col = df.pop('Start Day')
df.insert(4, col.name, col)

#move start month to the 5th index
col = df.pop('Start Month')
df.insert(5, col.name, col)


#change trip duration to minutes/hours
def convert(seconds):
    seconds = seconds % (24 * 60 * 60)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    return "%d:%02d:%02d" % (hour, minutes, seconds)


#df['Trip Duration'] = df['Trip Duration'].apply(convert)
df.tail()

In [ ]:
#create weekday/weekend column
df['Weekday/Weekend'] = df['Start Date'].dt.weekday.apply(lambda x: 'Weekday' if x <= 5 else 'Weekend')

df.head()